# Python私房手册-subprocess心得

## API

### subprocess.run

`subprocess.run`是python3.5以后加入的高层API，是首选的方法，函数签名为：
```python
subprocess.run(args, *, stdin=None, input=None, stdout=None, stderr=None, capture_output=False, shell=False, cwd=None, timeout=None, check=False, encoding=None, errors=None, text=None, env=None, universal_newlines=None, **other_popen_kwargs)
```
其中有一些参数不是很好理解，以下是基本的解释：
- stdin：定义子进程的输入，因为子进程和主进程的stdin,stdout,stderr是独立的，脚本和命令行解释器是在主进程里面执行，比如`print(input())`，此时是在主进程里面等待输入，如果在子进程里面等待输入，则需要将主进程的stdin传给子进程，比如：
```python
subprocess.run(["python", "-c", "print(f'{input()} is from main process.')"], stdin=sys.stdin)
```
`sys.stdin`是主进程的输入，`stdin=sys.stdin`表示子进程的输入就是当前主进程的输入，运行时当前程序会暂停，等待输入，运行结果如下：
```python
hello world!
hello world! is from main process.
CompletedProcess(args=['python', '-c', "print(f'{input()} is from main process.')"], returncode=0)
```
最后返回一个`CompletedProcess`对象。当然默认的stdin，stdout和stderr和主进程相同，不需要单独设置`stdin=sys.stdin`。
- check：一般情况下，当子进程执行程序出现错误，会抛出错误，但是当设置`capture_text`为True时，会捕获这些错误，存放在`CompletedProcess`对象的`stderr`属性里，可以调用`CompletedProcess`对象的`check_returncode()`方法重新抛出该错误。而当设置check参数为True时，此时会直接抛出错误，所以`check`参数总是和`capture_text`参数搭配使用。

### subprocess.check_output

`check_output()`就相当于`run(..., check=True, stdout=PIPE.stdout)`：
1. 子进程返回的returnCode如果不是0，则抛出`CalledProcessError`错误，错误会打印到主进程的stderr（即用户的屏幕）。
2. 子进程的stdout的输出通过管道返回给主进程，因此`check_output()`的返回值是子进程的打印内容。

在很多书上老是看到将`stderr`设置为`subprocess.STDOUT`，比如有如下的代码：
```python
r = subprocess.check_output("gecho hello;exit 0", shell=True)
```
此时会打印`/bin/sh: gecho: command not found`，返回值r为`b''`，因为命令gecho不对，但是由于returnCode为0，不会抛出`CalledProcessError`错误，而子进程的错误信息默认情况下会返回到主进程的stderr，即打印到屏幕上，而stdout由于什么都没有打印，所以返回`b''`。

如果想要捕获这个错误，则可以：
```python
r = subprocess.check_output("gecho hello;exit 0", shell=True, stderr=subprocess.STDOUT)
```
`subprocess.STDOUT`相当于子进程的`stdout`,因此这里相当于把子进程的`stderr`重定向到子进程的`stdout`，`stdout`的打印内容又通过`PIPE`返回给主进程，因此最后捕获了错误，r的结果为`/bin/sh: gecho: command not found`。

## 注意事项

### 命令中使用管道

使用`subprocess.run`时，如果命令中包含管道，比如`ps -ef | grep chrprocesser`命令，使用数组作为参数，会抛出错误，提示ps用法不对，如果加上`shell=True`，虽然可以返回结果，但是不正确，具体原因暂时未知。解决方法有好几种：
1. 直接使用字符串作为run的参数，注意，只要使用到管道，shell都需要设置为True：
```python
subprocess.run("ps -ef | grep chrprocesser", shell=True)
```
2. 使用Popen:

## 参考文章

- [How To Use subprocess to Run External Programs in Python 3](https://www.digitalocean.com/community/tutorials/how-to-use-subprocess-to-run-external-programs-in-python-3)